### Import Dataset
- Reads the dataset `game_statistics.csv` into a pandas DataFrame for analysis.

In [21]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

#CONSTS
READ_FOLDER_PATH = '../data/processed/'
WRITE_FOLDER_PATH = '../data/processed/'

data = pd.read_csv(READ_FOLDER_PATH+'game_statistics.csv')

print(data.shape[0])
pd.set_option("display.max_colwidth", None) 

941009


### Calculate League Averages

- Calculates league averages for: total_corners, home_corners and away_corners

In [22]:
def calc_league_corner_averages(df):
    """
    Calculate average statistics foreach league
    """
    league_stats = {}
    for game_id, game_data in df.groupby('id_odsp'):
        league=game_data['league'].iloc[0]
        country=game_data['country'].iloc[0]
        
        #Calculate full-time stats
        #note: the inneficiency of retrieving full-time corners twice is recognised, but this func is only run once so the inefficiency doesn't matter.
        home_corners_ft = game_data['home_corners'].iloc[-1]
        away_corners_ft = game_data['away_corners'].iloc[-1]
        total_corners_ft = home_corners_ft+away_corners_ft

        #Find corners at 80 minutes
        pre_80_data = game_data[game_data['time']<=80]
        if not pre_80_data.empty:
            home_corners_80 = pre_80_data['home_corners'].iloc[-1]
            away_corners_80 = pre_80_data['away_corners'].iloc[-1]
        else:
            home_corners_80 = 0
            away_corners_80 = 0
            
        home_corners_post80 =home_corners_ft-home_corners_80
        away_corners_post80 =away_corners_ft-away_corners_80
        total_corners_post80 = home_corners_post80+away_corners_post80
        
        #Store stats for league averages: corners, and specifically only home and away corners
        key = (country,league)
        if key not in league_stats:
            league_stats[key] = {'corners':[], 'home_corners':[], 'away_corners':[], 
                                 'post80_corners':[], 'post80_home_corners':[], 'post80_away_corners':[]
                            }
        
        league_stats[key]['corners'].append(total_corners_ft)
        league_stats[key]['home_corners'].append(home_corners_ft)
        league_stats[key]['away_corners'].append(away_corners_ft)
        league_stats[key]['post80_corners'].append(total_corners_post80)
        league_stats[key]['post80_home_corners'].append(home_corners_post80)
        league_stats[key]['post80_away_corners'].append(away_corners_post80)
    
    #Finally use stats to calculate averages for each league...
    league_averages = {}
    for key, stats in league_stats.items():
        country, league = key
        league_averages[key] = {
            'avg_corners': sum(stats['corners']) /len(stats['corners']) if stats['corners'] else 0,
            'avg_home_corners': sum(stats['home_corners'])/len(stats['home_corners']) if stats['home_corners'] else 0,
            'avg_away_corners': sum(stats['away_corners'])/len(stats['away_corners']) if stats['away_corners'] else 0,
            'avg_post80_corners': sum(stats['post80_corners']) / len(stats['post80_corners']) if stats['post80_corners'] else 0,
            'avg_post80_home_corners': sum(stats['post80_home_corners']) / len(stats['post80_home_corners']) if stats['post80_home_corners'] else 0,
            'avg_post80_away_corners': sum(stats['post80_away_corners']) / len(stats['post80_away_corners']) if stats['post80_away_corners'] else 0
        }
    
    return league_averages

### Process Game Data
- Groups match event data by game ID to get aggregated stats, including before the 80th minute, at full time, and in rolling windows (70-75, 75-80, 80-85 minutes).
- **Target variable** = whether a corner was won after 80 minutes

In [23]:
def process_game_data(df, league_averages):
    games = []
 
    # Group by game ID
    for game_id, game_data in df.groupby('id_odsp'):
        #Sort events by time
        game_data = game_data.sort_values(by='time')  

        #Get metadata
        id_odsp = game_id
        date = game_data['date'].iloc[0]
        season =game_data['season'].iloc[0] 
        league = game_data['league'].iloc[0]
        country= game_data['country'].iloc[0]
        home_team = game_data['ht'].iloc[0]
        away_team = game_data['at'].iloc[0]

        odd_h =game_data['odd_h'].iloc[0] 
        odd_d =game_data['odd_d'].iloc[0] 
        odd_a =game_data['odd_a'].iloc[0]

        #Split data at 80 mins
        pre_80_data = game_data[game_data['time'] <= 80]

        # Get score at 80 minutes
        score_pre_80 = pre_80_data['score'].iloc[-1]
        home_score_pre_80, away_score_pre_80 = map(int, score_pre_80.split('-'))
        # Get full-time score
        score_ft = game_data['score'].iloc[-1]

        # Calculate goal difference at 80 minutes
        goal_diff_80 = home_score_pre_80 - away_score_pre_80

        # Determine game state at 80 minutes
        if home_score_pre_80 > away_score_pre_80:
            game_state_80 = 'h'
        elif home_score_pre_80 < away_score_pre_80:
            game_state_80 = 'a'
        else:
            game_state_80 = 'd'

        #Pre-80 stats
        stats_pre_80 = {
            'home_shots_pre80': pre_80_data['home_shots'].iloc[-1], 
            'away_shots_pre80':pre_80_data['away_shots'].iloc[-1],
            'home_shots_on_target_pre80': pre_80_data['home_shots_on_target'].iloc[-1], 
            'away_shots_on_target_pre80': pre_80_data['away_shots_on_target'].iloc[-1],
            'home_shots_off_target_pre80':pre_80_data['home_shots_off_target'].iloc[-1],
            'away_shots_off_target_pre80': pre_80_data['away_shots_off_target'].iloc[-1],
            'home_yellow_cards_pre80': pre_80_data['home_yellow_cards'].iloc[-1],
            'away_yellow_cards_pre80':pre_80_data['away_yellow_cards'].iloc[-1], 
            'home_sending_off_pre80': pre_80_data['home_sending_off'].iloc[-1],
            'away_sending_off_pre80': pre_80_data['away_sending_off'].iloc[-1],
            'home_fouls_pre80':pre_80_data['home_fouls'].iloc[-1],
            'away_fouls_pre80': pre_80_data['away_fouls'].iloc[-1],
            'home_corners_pre80': pre_80_data['home_corners'].iloc[-1], 
            'away_corners_pre80': pre_80_data['away_corners'].iloc[-1],
        }

        #Full-time stats
        stats_ft = {  
            'home_shots_ft': game_data['home_shots'].iloc[-1], 
            'away_shots_ft':game_data['away_shots'].iloc[-1], 
            'home_shots_on_target_ft': game_data['home_shots_on_target'].iloc[-1], 
            'away_shots_on_target_ft': game_data['away_shots_on_target'].iloc[-1],
            'home_shots_off_target_ft': game_data['home_shots_off_target'].iloc[-1],
            'away_shots_off_target_ft': game_data['away_shots_off_target'].iloc[-1],
            'home_yellow_cards_ft':game_data['home_yellow_cards'].iloc[-1], 
            'away_yellow_cards_ft': game_data['away_yellow_cards'].iloc[-1],
            'home_sending_off_ft': game_data['home_sending_off'].iloc[-1],
            'away_sending_off_ft':game_data['away_sending_off'].iloc[-1],
            'home_fouls_ft': game_data['home_fouls'].iloc[-1],
            'away_fouls_ft': game_data['away_fouls'].iloc[-1],
            'home_corners_ft': game_data['home_corners'].iloc[-1], 
            'away_corners_ft':game_data['away_corners'].iloc[-1],
            'total_corners_ft': game_data['home_corners'].iloc[-1]+game_data['away_corners'].iloc[-1],
        }

        stats_pre_80_total = {
            "total_shots_pre_80":stats_pre_80["home_shots_pre80"] + stats_pre_80["away_shots_pre80"], 
            "total_fouls_pre_80": stats_pre_80["home_fouls_pre80"] +stats_pre_80["away_fouls_pre80"],
            "total_yellow_cards_pre_80": stats_pre_80["home_yellow_cards_pre80"] +stats_pre_80["away_yellow_cards_pre80"], 
            "total_sending_off_pre_80": stats_pre_80["home_sending_off_pre80"] + stats_pre_80["away_sending_off_pre80"],
            "total_corners_pre_80": stats_pre_80["home_corners_pre80"] + stats_pre_80["away_corners_pre80"], 
        }

        #Rolling windows for 70-75 and 75-80
        windows = { 
            '70_75': game_data[(game_data['time'] >= 70) & (game_data['time'] < 75)],
            '75_80': game_data[(game_data['time'] >= 75) & (game_data['time'] < 80)], 
        }
        window_stats = {}
        stats_columns = ['home_shots', 'away_shots', 'home_shots_on_target', 'away_shots_on_target',
                         'home_shots_off_target', 'away_shots_off_target', 'home_yellow_cards', 'away_yellow_cards',
                         'home_sending_off', 'away_sending_off', 'home_fouls', 'away_fouls',
                         'home_corners', 'away_corners']
        
        for window_name, window_data in windows.items():
            for stat in stats_columns:
                stat_change = (window_data[stat].iloc[-1] - window_data[stat].iloc[0]) if len(window_data) > 1 else 0
                window_stats[f'{stat}_{window_name}'] = stat_change


        stats_pre_80_total.update({
            "total_corners_70_75": window_stats["home_corners_70_75"] + window_stats["away_corners_70_75"],
            "total_shots_70_75": window_stats["home_shots_70_75"]+ window_stats["away_shots_70_75"], 
            "total_fouls_70_75": window_stats["home_fouls_70_75"] +window_stats["away_fouls_70_75"],
            "total_corners_75_80":window_stats["home_corners_75_80"] + window_stats["away_corners_75_80"],
            "total_shots_75_80": window_stats["home_shots_75_80"]+window_stats["away_shots_75_80"],
            "total_fouls_75_80": window_stats["home_fouls_75_80"] + window_stats["away_fouls_75_80"], 
        })

        #Corner difference (80 to FT)
        team_corner_diffs = {
            'home_corner_diff': stats_ft['home_corners_ft'] -stats_pre_80['home_corners_pre80'], 
            'away_corner_diff': stats_ft['away_corners_ft'] -stats_pre_80['away_corners_pre80'], 
        }

        corner_diff = team_corner_diffs['home_corner_diff'] + team_corner_diffs['away_corner_diff']

        # Calculate trailing team corners (corners won by the trailing team between 80 and full-time)
        trailing_team = None
        leading_team = None
        is_home_trailing = None 
        trailing_team_corners = None
        if home_score_pre_80 == away_score_pre_80 + 1: #away is trailing by 1 goal
            trailing_team = 'away'
            leading_team = 'home'
            is_home_trailing = 0
            trailing_team_corners = team_corner_diffs['away_corner_diff']
        elif away_score_pre_80 == home_score_pre_80 + 1: #home is trailing by 1 goal
            trailing_team = 'home'
            leading_team = 'away'
            is_home_trailing = 1
            trailing_team_corners = team_corner_diffs['home_corner_diff']

        #**************
        # Use league_averages to calc league stats
        key = (country,league)
        league_data = {
            #One-hot encodings for leagues...
            'league_england': 1 if country.lower()=='england' else 0,
            'league_spain': 1 if country.lower()=='spain' else 0,
            'league_germany': 1 if country.lower() =='germany' else 0, 
            'league_italy':1 if country.lower()=='italy' else 0,
            'league_france': 1 if country.lower()=='france' else 0,
             
            #League average stats:
            'league_avg_corners': league_averages.get(key, {}).get('avg_corners'),
            'league_avg_home_corners': league_averages.get(key, {}).get('avg_home_corners'), 
            'league_avg_away_corners': league_averages.get(key, {}).get('avg_away_corners'), 
            'league_avg_post80_corners': league_averages.get(key, {}).get('avg_post80_corners'),
            'league_avg_post80_home_corners': league_averages.get(key, {}).get('avg_post80_home_corners'), 
            'league_avg_post80_away_corners': league_averages.get(key, {}).get('avg_post80_away_corners')
             
            # ---> Corner rate comparisons... to measure game pace vs. expected league average pace
            # 'corner_rate_vs_avg': ((stats_pre_80['home_corners_pre80']+stats_pre_80['away_corners_pre80']) / 80) / 
            #                       (league_averages.get(key, {}).get('avg_corners') / 90),
            # 'home_corner_rate_vs_avg': (stats_pre_80['home_corners_pre80'] / 80) / 
            #                            (league_averages.get(key, {}).get('avg_home_corners') / 90),
            # 'away_corner_rate_vs_avg': (stats_pre_80['away_corners_pre80'] / 80) / 
            #                            (league_averages.get(key, {}).get('avg_away_corners') / 90)
        }
        #**************
 
        # Append processed data
        games.append({
            'id_odsp': id_odsp,
            'date': date, 
            'season': season,
            'league':league,
            'country': country, 
            'home_team': home_team,
            'away_team': away_team,
            'odd_h': odd_h,
            'odd_d': odd_d, 
            'odd_a': odd_a,
            'score_pre_80':score_pre_80,
            'score_ft': score_ft,
            'game_state_80': game_state_80,
            'goal_diff_80':goal_diff_80,
            **stats_pre_80, 
            **stats_pre_80_total,
            **stats_ft,
            **window_stats,
            **team_corner_diffs,
            **league_data,
            'corner_diff': corner_diff,
            'is_home_trailing': is_home_trailing,
            'trailing_team': trailing_team,
            'trailing_team_corners': trailing_team_corners,
        })

    # Convert to DataFrame
    return pd.DataFrame(games)

# Load dataset and process
league_averages = calc_league_corner_averages(data)
aggregated_data = process_game_data(data, league_averages) 

# Sort and reset index for clean output
aggregated_data = aggregated_data.sort_values(by=['date', 'id_odsp']).reset_index(drop=True)
aggregated_data

,id_odsp,date,season,league,country,home_team,away_team,odd_h,odd_d,odd_a,...,league_avg_corners,league_avg_home_corners,league_avg_away_corners,league_avg_post80_corners,league_avg_post80_home_corners,league_avg_post80_away_corners,corner_diff,is_home_trailing,trailing_team,trailing_team_corners
0,UFot0hit/,2011-08-05,2012,D1,germany,Borussia Dortmund,Hamburg SV,1.56,4.41,7.42,...,9.501866,5.307836,4.194030,1.302239,0.717662,0.584577,1,NaN,None,NaN
1,Aw5DflLH/,2011-08-06,2012,D1,germany,FC Augsburg,SC Freiburg,2.36,3.60,3.40,...,9.501866,5.307836,4.194030,1.302239,0.717662,0.584577,2,1.0,home,1.0
2,CzPV312a/,2011-08-06,2012,F1,france,Paris Saint-Germain,Lorient,1.55,4.50,9.40,...,9.479769,5.359345,4.120424,1.392582,0.767341,0.625241,4,1.0,home,3.0
3,GUOdmtII/,2011-08-06,2012,F1,france,Caen,Valenciennes,2.50,3.40,3.45,...,9.479769,5.359345,4.120424,1.392582,0.767341,0.625241,4,0.0,away,3.0
4,M7PhlM2C/,2011-08-06,2012,F1,france,Brest,Evian Thonon Gaillard,2.29,3.25,3.85,...,9.479769,5.359345,4.120424,1.392582,0.767341,0.625241,4,NaN,None,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9069,vJy048Er/,2017-01-22,2017,I1,italy,Empoli,Udinese,2.90,3.18,2.96,...,10.287572,5.747110,4.540462,1.517341,0.829480,0.687861,1,NaN,None,NaN
9070,xAkY8l6R/,2017-01-22,2017,I1,italy,Genoa,Crotone,1.97,4.35,8.00,...,10.287572,5.747110,4.540462,1.517341,0.829480,0.687861,2,NaN,None,NaN
9071,xSU9scI9/,2017-01-22,2017,E0,england,Chelsea,Hull,1.19,8.50,20.00,...,10.576597,5.874519,4.702079,1.585835,0.865281,0.720554,1,0.0,away,0.0
9072,xY7uZwOI/,2017-01-22,2017,F1,france,AS Monaco,Lorient,1.32,6.24,11.50,...,9.479769,5.359345,4.120424,1.392582,0.767341,0.625241,0,NaN,None,NaN


### Calculate Team Corner Rolling Averages

- Calculates rolling averages for corners, specifically: 
    - home vs away team
    - corners for vs against
    - total corners averages vs post 80 minute corner averages

In [24]:
def calc_team_corner_rolling_averages(df):
    team_corners = {}
    result_df = df.copy()
    
    for index, row in result_df.iterrows(): 
        season =row['season']
        home_team = row['home_team'] 
        away_team = row['away_team']
        
        home_corners_ft = row['home_corners_ft'] 
        away_corners_ft = row['away_corners_ft']
        home_corners_post80 = row['home_corner_diff']
        away_corners_post80 = row['away_corner_diff'] 

        league_avg_home_corners = row['league_avg_home_corners']
        league_avg_away_corners = row['league_avg_away_corners']
        league_avg_post80_home_corners = row['league_avg_post80_home_corners']
        league_avg_post80_away_corners = row['league_avg_post80_away_corners'] 
        
        #If either team are not in the dict... initialise entry:
        for team in [home_team, away_team]:
            if team not in team_corners or team_corners[team]['current_season'] != season:
                team_corners[team] = { 
                    'current_season': season,
                    'home_games': 0, 
                    'away_games': 0,

                    #ft stats:
                    'home_corners_for': league_avg_home_corners,
                    'home_corners_against': league_avg_away_corners,
                    'away_corners_for': league_avg_away_corners,
                    'away_corners_against': league_avg_home_corners,
                    'home_corners_for_avg':league_avg_home_corners,
                    'home_corners_against_avg': league_avg_away_corners, 
                    'away_corners_for_avg': league_avg_away_corners,
                    'away_corners_against_avg': league_avg_home_corners,

                    #post80 min stats:
                    'home_post80_corners_for': league_avg_post80_home_corners,
                    'home_post80_corners_against': league_avg_post80_away_corners,
                    'away_post80_corners_for': league_avg_post80_away_corners,
                    'away_post80_corners_against': league_avg_post80_home_corners,
                    'home_post80_corners_for_avg':league_avg_post80_home_corners,
                    'home_post80_corners_against_avg': league_avg_post80_away_corners, 
                    'away_post80_corners_for_avg': league_avg_post80_away_corners,
                    'away_post80_corners_against_avg': league_avg_post80_home_corners
                }

        #Save current rolling averages and game counts to the df
        result_df.loc[index,'home_games'] = team_corners[home_team]['home_games']
        result_df.loc[index,'home_corners_for_avg'] = team_corners[home_team]['home_corners_for_avg']
        result_df.loc[index,'home_corners_against_avg'] =team_corners[home_team]['home_corners_against_avg']
        result_df.loc[index,'home_post80_corners_for_avg'] = team_corners[home_team]['home_post80_corners_for_avg']
        result_df.loc[index,'home_post80_corners_against_avg'] =team_corners[home_team]['home_post80_corners_against_avg']

        result_df.loc[index,'away_games'] = team_corners[away_team]['away_games']
        result_df.loc[index,'away_corners_for_avg']= team_corners[away_team]['away_corners_for_avg'] 
        result_df.loc[index,'away_corners_against_avg'] = team_corners[away_team]['away_corners_against_avg']
        result_df.loc[index,'away_post80_corners_for_avg']= team_corners[away_team]['away_post80_corners_for_avg'] 
        result_df.loc[index,'away_post80_corners_against_avg'] = team_corners[away_team]['away_post80_corners_against_avg']
    
        #--- Update the rolling averages AFTER, i.e. ready for the next game ---

        #Home team: 
        team_corners[home_team]['home_games']+=1

        team_corners[home_team]['home_corners_for'] +=home_corners_ft 
        team_corners[home_team]['home_corners_against'] += away_corners_ft
        team_corners[home_team]['home_corners_for_avg'] =team_corners[home_team]['home_corners_for'] / (team_corners[home_team]['home_games']+1) 
        team_corners[home_team]['home_corners_against_avg'] = team_corners[home_team]['home_corners_against'] / (team_corners[home_team]['home_games']+1)

        team_corners[home_team]['home_post80_corners_for'] +=home_corners_post80
        team_corners[home_team]['home_post80_corners_against'] += away_corners_post80
        team_corners[home_team]['home_post80_corners_for_avg'] =team_corners[home_team]['home_post80_corners_for'] / (team_corners[home_team]['home_games']+1) 
        team_corners[home_team]['home_post80_corners_against_avg'] = team_corners[home_team]['home_post80_corners_against'] / (team_corners[home_team]['home_games']+1) 

        #Away team:
        team_corners[away_team]['away_games']+=1

        team_corners[away_team]['away_corners_for'] +=away_corners_ft
        team_corners[away_team]['away_corners_against'] += home_corners_ft 
        team_corners[away_team]['away_corners_for_avg'] = team_corners[away_team]['away_corners_for'] / (team_corners[away_team]['away_games']+1)
        team_corners[away_team]['away_corners_against_avg'] = team_corners[away_team]['away_corners_against'] / (team_corners[away_team]['away_games']+1)

        team_corners[away_team]['away_post80_corners_for'] +=away_corners_post80
        team_corners[away_team]['away_post80_corners_against'] += home_corners_post80
        team_corners[away_team]['away_post80_corners_for_avg'] = team_corners[away_team]['away_post80_corners_for'] / (team_corners[away_team]['away_games']+1)
        team_corners[away_team]['away_post80_corners_against_avg'] = team_corners[away_team]['away_post80_corners_against'] / (team_corners[away_team]['away_games']+1)
    
    return result_df 

aggregated_full = calc_team_corner_rolling_averages(aggregated_data)
aggregated_full 

,id_odsp,date,season,league,country,home_team,away_team,odd_h,odd_d,odd_a,...,home_games,home_corners_for_avg,home_corners_against_avg,home_post80_corners_for_avg,home_post80_corners_against_avg,away_games,away_corners_for_avg,away_corners_against_avg,away_post80_corners_for_avg,away_post80_corners_against_avg
0,UFot0hit/,2011-08-05,2012,D1,germany,Borussia Dortmund,Hamburg SV,1.56,4.41,7.42,...,0.0,5.307836,4.194030,0.717662,0.584577,0.0,4.194030,5.307836,0.584577,0.717662
1,Aw5DflLH/,2011-08-06,2012,D1,germany,FC Augsburg,SC Freiburg,2.36,3.60,3.40,...,0.0,5.307836,4.194030,0.717662,0.584577,0.0,4.194030,5.307836,0.584577,0.717662
2,CzPV312a/,2011-08-06,2012,F1,france,Paris Saint-Germain,Lorient,1.55,4.50,9.40,...,0.0,5.359345,4.120424,0.767341,0.625241,0.0,4.120424,5.359345,0.625241,0.767341
3,GUOdmtII/,2011-08-06,2012,F1,france,Caen,Valenciennes,2.50,3.40,3.45,...,0.0,5.359345,4.120424,0.767341,0.625241,0.0,4.120424,5.359345,0.625241,0.767341
4,M7PhlM2C/,2011-08-06,2012,F1,france,Brest,Evian Thonon Gaillard,2.29,3.25,3.85,...,0.0,5.359345,4.120424,0.767341,0.625241,0.0,4.120424,5.359345,0.625241,0.767341
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9069,vJy048Er/,2017-01-22,2017,I1,italy,Empoli,Udinese,2.90,3.18,2.96,...,10.0,4.431555,4.140042,0.893589,0.971624,9.0,4.454046,5.974711,0.968786,0.482948
9070,xAkY8l6R/,2017-01-22,2017,I1,italy,Genoa,Crotone,1.97,4.35,8.00,...,10.0,3.886101,4.321860,1.257225,0.517078,10.0,3.685497,9.613374,0.880715,1.166316
9071,xSU9scI9/,2017-01-22,2017,E0,england,Chelsea,Hull,1.19,8.50,20.00,...,10.0,5.806774,4.154734,0.624116,0.701869,10.0,5.427462,7.443138,0.701869,0.533207
9072,xY7uZwOI/,2017-01-22,2017,F1,france,AS Monaco,Lorient,1.32,6.24,11.50,...,10.0,5.487213,4.738220,0.706122,0.875022,10.0,5.374584,5.578122,1.238658,0.797031


### Save Processed Data
- Saves the processed dataset with all statistics to `proccessed_game_data.csv`.  
- A snippet of the first 500 rows is saved as `proccessed_game_data_snippet.csv`.  

In [25]:
print(aggregated_full.shape)

aggregated_full.to_csv(WRITE_FOLDER_PATH+'aggregated_full.csv', index=False)
aggregated_full.tail(500).to_csv(WRITE_FOLDER_PATH+'snippets/aggregated_full_snippet.csv', index=False)

(9074, 109)
